# Retention Score Test
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Mar 15, 2022

In [1]:
import boto3
import getpass

from datetime import datetime
from termcolor import colored, cprint
import pandas as pd
import awswrangler as wr
import pyodbc

In [2]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

LCTD-WRMQ
Successully logged into Start URL: https://stellaralgo.awsapps.com/start#/


### First we have to tell AWS which profile we'd like to login to, this will send us for a browser authentication trip:

### Connect to SQL Server:

In [3]:
try:
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
    client = boto3.client('redshift')

    conn = wr.data_api.redshift.connect(
        cluster_id="qa-app",
        database="datascience",
        db_user="admin"
    )
    
    print('CREDENTIALS RETRIEVED SUCCESSFULLY')
    
except Exception as err:
    print('FAILED TO RETRIEVE CREDENTIALS!', err)


CREDENTIALS RETRIEVED SUCCESSFULLY


### Create a dictionary of all teams that have retention scoring:

In [4]:
sqlcode =  f"""
    SELECT DISTINCT
        clientcode,
        lkupclientid,
        stlrdbname,
        leagueName
    FROM 
        datascience.ds.teamsconfig
    """
df = wr.data_api.redshift.read_sql_query(
            sql = sqlcode, 
            con = conn
        )

columns = df.columns.values

teams = []
for index, row in df.iterrows():
    teams.append(dict(zip(columns, row)))


print(f"Total teams: {len(teams)} ")
print(f"Example team {teams[0]} ")

Total teams: 38 
Example team {'clientcode': 'hops', 'lkupclientid': 9, 'stlrdbname': 'stlrmilb', 'leaguename': 'MILB'} 


In [6]:
for team in teams:
    
    if team['lkupclientid'] != 45: #exclude rays b/c they are in a diff db instance
        conn = wr.data_api.redshift.connect(
        cluster_id="qa-app",
        database= team['stlrdbname'],
        db_user="admin"
    )
        


        sql2 = f"""
                SELECT TOP (1) insertdate, count(*) as count
                FROM {team['stlrdbname']}.dw.customerretentionscores  
                WHERE [lkupclientid] = {team['lkupclientid']}
                GROUP BY insertDate
                ORDER BY convert(datetime, insertdate) DESC
            """

        df_customerretention = wr.data_api.redshift.read_sql_query(
            sql = sql2, 
            con = conn
        )

        results = []
        for row in df_customerretention.iterrows():
            team['insertdate'] = row[0]
            team['count'] = row[1]
        
print(f"Example team {teams[0]} ")

RedshiftDataApiFailedException: Request 79873841-afc8-4799-9d81-c82b69d84665 failed with status FAILED and error ERROR: syntax error at or near "("
  Position: 29

### Add two more fields to each teams dictionary (insertdate & count):

In [93]:
cursor = CNXN.cursor()

for team in teams:

    sql = f"""
            SELECT TOP (1) insertdate, count(*) as count
            FROM [DataScience].[DS].[customerScores]
            WHERE [lkupclientid] = {team['lkupclientid']}
            GROUP BY insertDate
            ORDER BY convert(datetime, insertdate) DESC
        """
    
    cursor = CNXN.cursor().execute(sql)

    results = []
    for row in cursor.fetchall():
        team['insertdate'] = row[0]
        team['count'] = row[1]

print(f"Example team {teams[0]} ")

Example team {'clientcode': '66ers', 'lkupclientid': 19, 'stlrDBName': 'stlrMILB', 'leagueName': 'MILB', 'insertdate': '03-15-2022 10:26:15', 'count': 255} 


### Write a test to see if insertdate is greater than todays date:

In [94]:
today = datetime.now().strftime("%m-%d-%Y 00:00:00")

print("SCORE REPORT FOR DATASCIENCE customerScores TABLE:")
for team in teams:
    
    if team['insertdate'] > today:
        print(colored(f" > {team['count']} Scores Updated: {team['clientcode']} ({team['lkupclientid']})", 'green'))
    else:
        print(colored(f" ! {team['count']} Scores NOT Updated: {team['clientcode']} ({team['lkupclientid']})", 'red'))

SCORE REPORT FOR DATASCIENCE customerScores TABLE:
 ! 255 Scores NOT Updated: 66ers (19)
 ! 1634 Scores NOT Updated: admirals (16)
 ! 5622 Scores NOT Updated: blazers (5)
 ! 3265 Scores NOT Updated: bulls (11)
 ! 5130 Scores NOT Updated: canucks (7)
 ! 2648 Scores NOT Updated: coyotes (55)
 ! 2296 Scores NOT Updated: drive (48)
 ! 5187 Scores NOT Updated: dynamo (13)
 ! 931 Scores NOT Updated: elpaso (30)
 ! 325 Scores NOT Updated: fireflies (28)
 ! 5381 Scores NOT Updated: flames (36)
 ! 7936 Scores NOT Updated: galaxy (6)
 > 105 Scores Updated: grizzlies (27)
 ! 429 Scores NOT Updated: hartfordyardgoats (44)
 ! 777 Scores NOT Updated: hitmen (37)
 ! 931 Scores NOT Updated: hops (9)
 ! 249 Scores NOT Updated: kanecounty (51)
 ! 16 Scores NOT Updated: legends (26)
 ! 603 Scores NOT Updated: loons (20)
 ! 3686 Scores NOT Updated: nhlpanthers (93)
 ! 1525 Scores NOT Updated: okcdodgers (47)
 ! 842 Scores NOT Updated: rainiers (17)
 ! 415 Scores NOT Updated: rattlers (24)
 ! 264 Scores NO

### Add another field to each team (dimdateid) from each teams STLR table:

In [80]:
for team in teams:
    
    if team['lkupclientid'] != 45: #exclude rays b/c they are in a diff db instance
    
        CNXN = get_sql_connection(team['stlrDBName'])
        cursor = CNXN.cursor()

        sql = f"""
                SELECT TOP (1) dimdateid
                FROM {team['stlrDBName']}.DW.factCustomerRetentionScore 
                WHERE [lkupclientid] = {team['lkupclientid']}
                GROUP BY dimdateid
                ORDER BY dimdateid DESC
            """

        cursor = CNXN.cursor().execute(sql)

        results = []
        for row in cursor.fetchall():
            team['dimdateid'] = row[0]
        
print(f"Example team {teams[0]} ")

Example team {'clientcode': '66ers', 'lkupclientid': 19, 'stlrDBName': 'stlrMILB', 'leagueName': 'MILB', 'insertdate': '03-15-2022 10:26:15', 'count': 255, 'dimdateid': 20220107} 


In [89]:
today = int(datetime.now().strftime("%Y%m%d"))

print("SCORE REPORT FOR STLR factCustomerRetentionScore TABLE:")
for team in teams:
    
    if team['lkupclientid'] != 45: #exclude rays b/c they are in a diff db instance

        if team['dimdateid'] >= today:
            print(colored(f" > {team['count']} Scores Updated: {team['clientcode']} ({team['lkupclientid']})", 'green'))
        else:
            print(colored(f" ! {team['count']} Scores NOT Updated: {team['clientcode']} ({team['lkupclientid']})", 'red'))

SCORE REPORT FOR STLR faceCustomerRetentionScore TABLE:
 ! 255 Scores NOT Updated: 66ers (19)
 ! 1634 Scores NOT Updated: admirals (16)
 > 5622 Scores Updated: blazers (5)
 ! 3265 Scores NOT Updated: bulls (11)
 > 5130 Scores Updated: canucks (7)
 > 2648 Scores Updated: coyotes (55)
 ! 2296 Scores NOT Updated: drive (48)
 > 5187 Scores Updated: dynamo (13)
 ! 931 Scores NOT Updated: elpaso (30)
 ! 325 Scores NOT Updated: fireflies (28)
 > 5381 Scores Updated: flames (36)
 > 7936 Scores Updated: galaxy (6)
 ! 105 Scores NOT Updated: grizzlies (27)
 ! 429 Scores NOT Updated: hartfordyardgoats (44)
 > 777 Scores Updated: hitmen (37)
 ! 931 Scores NOT Updated: hops (9)
 ! 249 Scores NOT Updated: kanecounty (51)
 ! 16 Scores NOT Updated: legends (26)
 ! 603 Scores NOT Updated: loons (20)
 ! 3686 Scores NOT Updated: nhlpanthers (93)
 ! 1525 Scores NOT Updated: okcdodgers (47)
 ! 842 Scores NOT Updated: rainiers (17)
 ! 415 Scores NOT Updated: rattlers (24)
 ! 1648 Scores NOT Updated: renoace